<a href="https://colab.research.google.com/github/marco-siino/DA-ESWA/blob/main/code/augmentation/MBart_HSS_TrainingSet_Augmentation_DE_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing modules.

In [1]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00


In [2]:
import os
import os.path
from os import path
import re
import shutil
import string
import tensorflow as tf

from io import open
from pathlib import Path
from google.colab import files

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [3]:
!pip install xmltodict
import xmltodict 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing DS and extract in current working directory.

In [4]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
train_set_url = 'https://github.com/marco-siino/DA-ESWA/raw/main/data/hss/hss-training-cleaned.zip'  # train FNS
train_set_path = tf.keras.utils.get_file("pan21-author-profiling-training-2021-03-14-augmented.zip", train_set_url,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

train_set_dir = os.path.join(os.path.dirname(train_set_path), 'pan21-author-profiling-training-2021-03-14-augmented.zip')

print(train_set_path)
print(train_set_dir)

1247321/1247321 [==============================] - 0s 0us/step
./pan21-author-profiling-training-2021-03-14-augmented.zip
./pan21-author-profiling-training-2021-03-14-augmented.zip


## Function to pre-process source text.

In [5]:
def custom_standardization(text):
  tag_open_CDATA_removed = re.sub('<\!\[CDATA\[', ' ',text)
  tag_closed_CDATA_removed = re.sub('\]{1,}>', ' ',tag_open_CDATA_removed)
  tag_author_lang_en_removed = re.sub('<author lang="en">', ' ',tag_closed_CDATA_removed)
  tag_author_lang_en_removed = re.sub('<author lang="en"', ' ',tag_closed_CDATA_removed)
  #only for test remove if needed
  tag_author_lang_en_removed = re.sub('class="0">', ' ', tag_author_lang_en_removed )
  tag_author_lang_en_removed = re.sub('class="1">', ' ', tag_author_lang_en_removed )
  ######
  tag_closed_author_removed = re.sub('</author>', '', tag_author_lang_en_removed)
  tag_closed_author_removed = re.sub('</author', '', tag_closed_author_removed)
  tag_open_documents_removed = re.sub('<documents>\n(\t){0,2}', '',tag_closed_author_removed)
  tag_open_documents_removed = re.sub('<document>', '',tag_open_documents_removed)
  tag_closed_documents_removed = re.sub('</documents>\n(\t){0,2}', ' ',tag_open_documents_removed)
  lowercased_text = tag_closed_documents_removed.lower()
  return lowercased_text

## Data Augmentation function.

In [6]:
def chunkstring(string, length):
  res = list((string[0+i:length+i] for i in range(0, len(string), length)))
  return res

def enhance_one_sample(sample, TARGET='it', return_both=True):  
  preprocessed_text = custom_standardization(sample)

  #chunk to avoid character limits  
  TOBETRANS = chunkstring(preprocessed_text, 4000)
  translated_it = GoogleTranslator(source='en', target=TARGET).translate_batch(TOBETRANS)
  reversed_trans = GoogleTranslator(source=TARGET, target='en').translate_batch(translated_it)
  merged_chunks =''.join(reversed_trans)
  enhanced_sample = preprocessed_text+merged_chunks
  if return_both == False:
    enhanced_sample = merged_chunks
  return enhanced_sample

In [7]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
config = model,confi
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [11]:
def translate_en_lang(article_en, target_lang):
  preprocessed_text = custom_standardization(article_en)
  print('preprocessed_text type', type(preprocessed_text))
  print('\n\npreprocessed_text', preprocessed_text)
  if target_lang == 'de':
    #############################
    # translate English to German
    ############################# 
    tokenizer.src_lang = "en_XX"
    encoded_en = tokenizer(preprocessed_text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_en,
        forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"],
        max_new_tokens=30000
        )
    translated_de = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    translated_de = translated_de[0]
    #print('\n translated_de',translated_de)

    #############################
    #Backtranslate German to English
    #############################
    tokenizer.src_lang = "de_DE"
    encoded_de = tokenizer(translated_de, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_de,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
        )
    back_translated_en = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    back_translated_en = back_translated_en[0]
    #print('\n back_translated_DE_en',back_translated_en)
  elif target_lang == 'it':
    #############################
    # translate English to Italian 
    #############################
    tokenizer.src_lang = "en_XX"
    encoded_en = tokenizer(preprocessed_text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_en,
        forced_bos_token_id=tokenizer.lang_code_to_id["it_IT"]
        )
    translated_it = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    translated_it = translated_it[0]
    #print('\n translated_it',translated_it)

    #############################
    #Backtranslate Italian to English
    #############################
    tokenizer.src_lang = "it_IT"
    encoded_it = tokenizer(translated_it, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_it,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
        )
    back_translated_en = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    back_translated_en = back_translated_en[0]
    #print('\n back_translated_IT_en', back_translated_en)
  enhanced_sample = preprocessed_text + '' + back_translated_en
  return enhanced_sample

# Generating augmented train set.

In [9]:
## Generate the augmented training set as a zip file.

# pan22-author-profiling-training-2022-03-29-augmented.zip has to be created.
if not os.path.exists('pan21-author-profiling-training-2021-03-14-augmented'):
    os.makedirs('pan21-author-profiling-training-2021-03-14-augmented')
if not os.path.exists('pan21-author-profiling-training-2021-03-14-augmented/en/'):
    os.makedirs('pan21-author-profiling-training-2021-03-14-augmented/en/')

In [10]:
# Copy the ground truth file.
if os.path.exists('/content/pan21-author-profiling-training-2021-03-14-augmented'):
        shutil.copyfile('/content/pan21-author-profiling-training-2021-03-14-augmented/en/truth.txt', '/content/pan21-author-profiling-training-2021-03-14-augmented/truth.txt')

In [ ]:
file_nr = 0
count_errors = 0
for filename in os.listdir('pan21-author-profiling-training-2021-03-14-augmented/en/'):
  file_nr += 1
  x = filename.split(".")      
  author_id = x[0]
  print("File nr.:", file_nr)
  print("Filename:", filename)
  text = open('pan21-author-profiling-training-2021-03-14-augmented/en/'+filename, 'r').read()
  print('text', type(text))
  enhanced_sample = translate_en_lang("The golden sun set over the tranquil ocean as the gentle breeze brushed against my faceThe golden sun set over the tranquil ocean as the gentle breeze brushed against my faceThe golden sun set over the tranquil ocean as the gentle breeze brushed against my faceThe golden sun set over the tranquil ocean as the gentle breeze brushed against my faceThe golden sun set over the tranquil ocean as the gentle breeze brushed against my faceThe golden sun set over the tranquil ocean as the gentle breeze brushed against my face.", target_lang='it')
  # try:
  #   enhanced_sample = translate_en_lang(text, target_lang='de')
  #   print('enhanced_sample')
  #   # f = open("pan21-author-profiling-training-2021-03-14-augmented/en/"+author_id+".xml", "a")
  #   # f.write(enhanced_sample)
  #   # f.close()
  #   print('Succes')
  # except:
  #   print('!!! FAILED !!!!')
  #   count_errors += 1
  #   # f = open("pan21-author-profiling-training-2021-03-14-augmented/en/"+author_id+".xml", "a")
  #   # preprocessed_text = custom_standardization(text)
  #   # f.write(preprocessed_text)
  #   # f.close()
print('cunt err', count_errors)
# !zip -r pan21-author-profiling-training-2021-03-14-augmented.zip pan21-author-profiling-training-2021-03-14-augmented
# # If automatic download doesn't start, open the directory browser on the left menu and download the zip file manually.
# files.download("pan21-author-profiling-training-2021-03-14-augmented.zip")

File nr.: 1
Filename: b4030e7f9f049a602defab723599fdeb.xml
text <class 'str'>
preprocessed_text type <class 'str'>


preprocessed_text the golden sun set over the tranquil ocean as the gentle breeze brushed against my facethe golden sun set over the tranquil ocean as the gentle breeze brushed against my facethe golden sun set over the tranquil ocean as the gentle breeze brushed against my facethe golden sun set over the tranquil ocean as the gentle breeze brushed against my facethe golden sun set over the tranquil ocean as the gentle breeze brushed against my facethe golden sun set over the tranquil ocean as the gentle breeze brushed against my face.
File nr.: 2
Filename: d47a4b83cd692b1c0c5e8444bf1bc2a5.xml
text <class 'str'>
preprocessed_text type <class 'str'>


preprocessed_text the golden sun set over the tranquil ocean as the gentle breeze brushed against my facethe golden sun set over the tranquil ocean as the gentle breeze brushed against my facethe golden sun set over the tran